In [2]:
%pip install torch
%pip install transformers[torch]
%pip install ipywidgets
%pip install datasets
%pip install -U scikit-learn
%pip install -U ipywidgets
%pip install 'accelerate>=1.1.0'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from itertools import product
from collections import defaultdict
from torch.nn import functional as F


pd.set_option('display.width',1000)


In [4]:
torch.cuda.empty_cache()

In [5]:
!nvidia-smi

Tue Feb 24 16:16:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.06              Driver Version: 580.65.06      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:21:00.0  On |                  N/A |
| 56%   75C    P2            208W /  250W |   10512MiB /  11264MiB |     92%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
RANDOM_SEED = 1
torch.manual_seed(RANDOM_SEED)

EPOCHS = 3
MAX_LENGTH = 128
K = 5


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
from load_data import DontPatronizeMe

DATA_DIR = 'data/'
TEST_PATH = f'{DATA_DIR}task4_test.tsv'

dpm = DontPatronizeMe(DATA_DIR, TEST_PATH)
dpm.load_task1()

data = dpm.train_task1_df

trids = pd.read_csv(f'{DATA_DIR}train_semeval_parids-labels.csv')
devids = pd.read_csv(f'{DATA_DIR}dev_semeval_parids-labels.csv')

trids['par_id'] = trids.par_id.astype(str)
devids['par_id'] = devids.par_id.astype(str)

cols = ['par_id', 'text', 'label_y']

trdf = trids.merge(data, on='par_id', how='left')[cols]
devdf = devids.merge(data, on='par_id', how='left')[cols]

# rename label_y to label for Trainer
trdf.rename(columns={"label_y": "labels"}, inplace=True)
devdf.rename(columns={"label_y": "labels"}, inplace=True)


print(trdf.head())
print()

print(devdf.head())
print()

trds = Dataset.from_pandas(trdf)
print(trds)


NameError: name 'pd' is not defined

In [5]:
counts = trdf['labels'].value_counts()
print("Original class distribution:")
print(counts)
print()

weights = torch.tensor([counts[0]/counts[0], counts[0]/counts[1]], dtype=torch.float).to(device)
print("Class weights:", weights)
print()


Original class distribution:
labels
0    7581
1     794
Name: count, dtype: int64

Class weights: tensor([1.0000, 9.5479], device='cuda:0')



In [6]:
from sklearn.model_selection import StratifiedKFold, train_test_split

VAL_SIZE = 0.1

trdf, valdf = train_test_split(trdf, test_size=VAL_SIZE, random_state=RANDOM_SEED, stratify=trdf['labels'])

skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=RANDOM_SEED)

X = trdf["text"].values
y = trdf["labels"].values

folds = list(skf.split(X, y))


In [7]:
from sklearn.metrics import f1_score

class BalancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, 2)

        loss_fn = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    f1 = f1_score(labels, preds, pos_label=1)
    return { "f1": f1 }



In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

rob_checkpoint = "FacebookAI/roberta-base"
deb_checkpoint = "microsoft/deberta-base"

rob_tokenizer = AutoTokenizer.from_pretrained(rob_checkpoint)
deb_tokenizer = AutoTokenizer.from_pretrained(deb_checkpoint)

def tokenize(tokenizer, batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

architectures = [("Roberta", rob_checkpoint, rob_tokenizer), ("Deberta", deb_checkpoint, deb_tokenizer)]


In [2]:
!nvidia-smi

Tue Feb 24 15:57:53 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.06              Driver Version: 580.65.06      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:21:00.0  On |                  N/A |
| 58%   77C    P2            232W /  250W |    8153MiB /  11264MiB |     99%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
LRS = [2e-5, 3e-5]
BSZS = [16, 32]
WEIGHT_DECAY = 0.05
DROPOUT = 0.2

num_epochs = len(folds) * len(architectures) * len(LRS) * len(BSZS) * EPOCHS
print(f"Total training epochs: {num_epochs}")
print()

# Determine best hyperparameters using k-fold cross-validation
def hyperparameter_search():
    hp_metrics = {}
    hp_metrics["Roberta"] = defaultdict(float)
    hp_metrics["Deberta"] = defaultdict(float)

    res = {}

    for name, checkpoint, tokenizer in architectures:
        print(f"Training {name}...")
        print()

        for fold, (train_idx, val_idx) in enumerate(folds):
            print(f"Fold {fold + 1}/{K}")

            trds = Dataset.from_pandas(trdf.iloc[train_idx])
            valds = Dataset.from_pandas(trdf.iloc[val_idx])

            trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
            valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

            trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
            valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

            for lr, bsz in product(LRS, BSZS):
                model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                                           hidden_dropout_prob=DROPOUT,
                                                                           attention_probs_dropout_prob=DROPOUT).to(device)
                training_args = TrainingArguments(
                    learning_rate=lr,
                    per_device_train_batch_size=bsz,
                    per_device_eval_batch_size=bsz,
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    eval_strategy="epoch",
                    save_strategy="no",
                    remove_unused_columns=False,
                    weight_decay=WEIGHT_DECAY,
                )

                trainer = BalancedTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=trds,
                    eval_dataset=valds,
                    compute_metrics=compute_metrics
                )

                trainer.train()
                metrics = trainer.evaluate()

                model = model.cpu()

                hp_metrics[name][(lr, bsz)] += metrics["eval_f1"]
            
        best_hp = max(hp_metrics[name], key=hp_metrics[name].get)
        avg_f1 = hp_metrics[name][best_hp] / K
        print(f"Best hyperparameters for {name}: {best_hp}, Average F1: {avg_f1:.4f}")
        res[name] = {
            "best_hps": best_hp,
            "avg_f1": avg_f1
        }

    return res

Total training epochs: 120



In [ ]:
tuning_res = hyperparameter_search()
print(tuning_res)

Training Roberta...

Fold 1/5


Map:   0%|          | 0/6029 [00:00<?, ? examples/s]

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

Map:   0%|          | 0/6029 [00:00<?, ? examples/s]

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

DebertaForSequenceClassification LOAD REPORT from: microsoft/deberta-base
Key                                     | Status     | 
----------------------------------------+------------+-
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED | 
lm_predictions.lm_head.dense.weight     | UNEXPECTED | 
lm_predictions.lm_head.dense.bias       | UNEXPECTED | 
lm_predictions.lm_head.bias             | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED | 
pooler.dense.weight                     | MISSING    | 
classifier.bias                         | MISSING    | 
pooler.dense.bias                       | MISSING    | 
classifier.weight                       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Epoch,Training Loss,Validation Loss


In [10]:
tuning_res = {
    "Roberta": {
        "best_hps": (3e-5, 32),
        "avg_f1": 0.5557551489307914
    },
    "Deberta": {
        "best_hps": (2e-5, 32),
        "avg_f1": 0.5562773611889518
    }
}

In [ ]:
FINAL_EPOCHS = 5 
THRESHOLDS = [0.3, 0.4, 0.5, 0.6, 0.7]

NUM_MODELS_PER_ARCH = 2

# Retrain best models on full training set
def train_ensemble_models(tuning_results):
    ensemble_models = {}

    for name, checkpoint, tokenizer in architectures:
        print(f"Retraining {name} on full training set...")

        trds = Dataset.from_pandas(trdf)
        valds = Dataset.from_pandas(valdf)

        trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

        lr, bsz = tuning_results[name]["best_hps"]

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                                   hidden_dropout_prob=DROPOUT,
                                                                   attention_probs_dropout_prob=DROPOUT).to(device)
        training_args = TrainingArguments(
            output_dir=f'./final_results/{name}',
            learning_rate=lr,
            per_device_train_batch_size=bsz,
            per_device_eval_batch_size=bsz,
            num_train_epochs=FINAL_EPOCHS,
            logging_dir=f'./final_logs/{name}',
            eval_strategy="epoch",
            logging_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            weight_decay=WEIGHT_DECAY
        )

        trainer = BalancedTrainer(
            model=model,
            args=training_args,
            train_dataset=trds,
            eval_dataset=valds,
            compute_metrics=compute_metrics
        )

        trainer.train()
        metrics = trainer.evaluate()

        model = model.to("cpu")

        ensemble_models[name] = (model, tokenizer, metrics["eval_f1"])
    
    tot_f1 = sum(ensemble_models[name][2] for name in ensemble_models)
    for name in ensemble_models:
        model, tokenizer, f1 = ensemble_models[name]
        ensemble_weight = f1 / tot_f1
        ensemble_models[name] = (model, tokenizer, ensemble_weight)
    
    return ensemble_models


In [12]:
ensemble_models = train_ensemble_models(tuning_res)
print(ensemble_models)

Retraining Roberta on full training set...


Map:   0%|          | 0/7956 [00:00<?, ? examples/s]

Map:   0%|          | 0/419 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['deberta.embeddings.LayerNorm.weight', 'deberta.embeddings.LayerNorm.bias', 'deberta.encoder.layer.0.attention.output.LayerNorm.weight', 'deberta.encoder.layer.0.attention.output.LayerNorm.bias', 'deberta.encoder.layer.0.output.LayerNorm.weight', 'deberta.encoder.layer.0.output.LayerNorm.bias', 'deberta.encoder.layer.1.attention.output.LayerNorm.weight', 'deberta.encoder.layer.1.attention.output.LayerNorm.bias', 'deberta.encoder.layer.1.output.LayerNorm.weight', 'deberta.encoder.layer.1.output.LayerNorm.bias', 'deberta.encoder.layer.2.attention.output.LayerNorm.weight', 'deberta.encoder.layer.2.attention.output.LayerNorm.bias', 'deberta.encoder.layer.2.output.LayerNorm.weight', 'deberta.encoder.layer.2.output.LayerNorm.bias', 'deberta.encoder.layer.3.attention.output.LayerNorm.weight', 'deberta.encoder.layer.3.attention.output.LayerNorm.bias', 'deberta.encoder.layer.3.output.LayerNorm.weight', 'deberta.encoder.layer.3.output.Laye

{'Roberta': (RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


In [32]:
class UnbalancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, 2)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
# from sklearn.model_selection import train_test_split

# FINAL_EPOCHS = 5 
# HOLDOUT = 0.05
# WEIGHT_DECAY = 0.05
# DROPOUT = 0.2

# def train_unbalanced(tuning_results):
#     # Split into train and val sets for monitoring best checkpoint
#     trdf_train, trdf_val = train_test_split(trdf, test_size=HOLDOUT, random_state=RANDOM_SEED, stratify=trdf['labels'])

#     name, checkpoint, tokenizer = architectures[0]

#     print(f"Training {name} on full training set... [unweighted loss]")

#     trds = Dataset.from_pandas(trdf_train)
#     valds = Dataset.from_pandas(trdf_val)

#     trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
#     valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
#     trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
#     valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

#     lr, bsz = tuning_results[name]["best_hps"]

#     model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
#                                                                 hidden_dropout_prob=DROPOUT,
#                                                                 attention_probs_dropout_prob=DROPOUT).to(device)
#     training_args = TrainingArguments(
#         output_dir=f'./unbalanced/{name}',
#         learning_rate=lr,
#         per_device_train_batch_size=bsz,
#         per_device_eval_batch_size=bsz,
#         num_train_epochs=FINAL_EPOCHS,
#         eval_strategy="epoch",
#         logging_strategy="epoch",
#         save_strategy="epoch",
#         save_total_limit=1,
#         load_best_model_at_end=True,
#         metric_for_best_model="f1",
#         weight_decay=WEIGHT_DECAY
#     )

#     trainer = UnbalancedTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=trds,
#         eval_dataset=valds,
#         compute_metrics=compute_metrics
#     )

#     trainer.train()
#     metrics = trainer.evaluate()

#     model = model.to("cpu")

#     print(f"Positive class F1: {metrics['eval_f1']}")

#     return model


In [36]:
unbalanced_model = train_unbalanced(tuning_res)

Training Roberta on full training set... [unweighted loss]


Map:   0%|          | 0/7956 [00:00<?, ? examples/s]

Map:   0%|          | 0/419 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Positive class F1: 0.5526315789473685


In [ ]:
from torch.utils.data import DataLoader

BSZ=16

def ensemble_predict(dataset, ensemble_models):
    dataloader = DataLoader(dataset, batch_size=BSZ)
    all_preds = []

    for batch in dataloader:
        texts = batch["text"]
        batch_probs = None

        for model, tokenizer, weight in ensemble_models.values():
            # Tokenize this batch for this model
            encodings = tokenizer(
                texts,
                padding=True,
                truncation=True,
                return_tensors="pt"
            )
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)

            model.to(device)
            model.eval()
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                probs = F.softmax(outputs.logits, dim=-1) * weight

            # Accumulate weighted probabilities
            if batch_probs is None:
                batch_probs = probs
            else:
                batch_probs += probs

            # Move model back to CPU to free GPU memory
            model.to("cpu")

        # Final predictions for this batch
        preds = torch.argmax(batch_probs, dim=-1)
        all_preds.extend(preds.cpu().numpy())

    return all_preds

In [18]:
# Helper function to save predictions to an output file
def labels2file(p, outf_path):
    with open(outf_path, "w") as f:
        for pred in p:
            f.write(f"{pred}\n")

In [ ]:
import os

valds = Dataset.from_pandas(valdf)

fake_models = {
    "Roberta": (unbalanced_model, rob_tokenizer, 1),
}

# Generate dev set predictions using ensemble
preds = ensemble_predict(valds, fake_models)
labels2file(preds, os.path.join('res/', 'dev.txt'))

# Save reference labels for dev set
labels2file(valdf['labels'].tolist(), os.path.join('ref/', 'dev.txt'))

In [38]:
# Unbalanced Roberta

!python3 evaluation.py . . dev.txt
!cat scores.txt

task1_precision:0.49808429118773945
task1_recall:0.6532663316582915
task1_f1:0.5652173913043478


In [ ]:
import os

valds = Dataset.from_pandas(valdf)

# For ablation studies, try models separately
models = {}
models["Roberta"] = ensemble_models["Roberta"]
models["Deberta"] = ensemble_models["Deberta"]

# Generate dev set predictions using ensemble
preds = ensemble_predict(valds, models)
labels2file(preds, os.path.join('res/', 'dev.txt'))

# Save reference labels for dev set
labels2file(valdf['labels'].tolist(), os.path.join('ref/', 'dev.txt'))


In [ ]:
# Just Deberta

# !python3 evaluation.py . . dev.txt
# !cat scores.txt

task1_precision:0.4014423076923077
task1_recall:0.8391959798994975
task1_f1:0.5430894308943089


In [ ]:
# Just Roberta

# !python3 evaluation.py . . dev.txt
# !cat scores.txt

task1_precision:0.4114441416893733
task1_recall:0.7587939698492462
task1_f1:0.5335689045936396


In [24]:
!python3 evaluation.py . . dev.txt
!cat scores.txt

task1_precision:0.4074074074074074
task1_recall:0.7738693467336684
task1_f1:0.5337954939341422


In [21]:
dpm.load_test()

cols = ['par_id', 'text', 'label']
testdf = dpm.test_set_df.copy()
testdf = testdf[cols]
testdf.rename(columns={"label": "labels"}, inplace=True)
print(testdf.head())

testds = Dataset.from_pandas(testdf)

  par_id                                               text  labels
0    t_4  Members of the church , which is part of Ken C...       1
1    t_5  "To ensure that "" Priority Agriculture Progra...       1
2    t_6  The deportees stepped off their flight from El...       1
3    t_7  PIMS staffer who raped disabled girl at ICU wa...       1
4    t_9  I conclude , Yes , the general FEELING generat...       1


In [ ]:
# Generate test set predictions
preds = ensemble_predict(testds, ensemble_models)
labels2file(preds, os.path.join('res/', 'test.txt'))